In [1]:
# compute_solutionrs/dynamic_programming_compute_solutionr.py

import numpy as np
from typing import List, Dict, Tuple


class DPSolver:
    """
    Dynamic Programming compute_solutionr for the TSP.
    Computes the shortest path visiting all cities exactly once.
    """

    def __init__(self, n_cities: int, dist_matrix: np.ndarray):
        self.n_cities = n_cities
        self.dist_matrix = dist_matrix
        self.cost_table = {}
        self.route_policy = {}
        self.total_min_cost = None
        self.last_city = None

    def compute_solution(self):
        """Solves the TSP using dynamic programming."""
        num_subsets = 1 << self.n_cities  # Total number of subsets

        # Initialize the value table with infinity
        for subset in range(num_subsets):
            for last in range(self.n_cities):
                self.cost_table[(subset, last)] = np.inf
        self.cost_table[(1 << 0, 0)] = 0  # Starting from city 0

        # Dynamic programming to fill the value table
        for subset_size in range(2, self.n_cities + 1):
            for subset in [s for s in range(num_subsets) if bin(s).count('1') == subset_size and s & (1 << 0)]:
                for last in range(self.n_cities):
                    if not (subset & (1 << last)):
                        continue
                    prev_subset = subset ^ (1 << last)
                    if prev_subset == 0:
                        continue
                    for k in range(self.n_cities):
                        if (prev_subset & (1 << k)):
                            cost = self.cost_table[(prev_subset, k)] + self.dist_matrix[k, last]
                            if cost < self.cost_table[(subset, last)]:
                                self.cost_table[(subset, last)] = cost
                                self.route_policy[(subset, last)] = k

        # Close the tour by returning to the starting city
        full_subset = (1 << self.n_cities) - 1
        min_cost = np.inf
        last_city = None
        for k in range(self.n_cities):
            if k == 0:
                continue
            cost = self.cost_table[(full_subset, k)] + self.dist_matrix[k, 0]
            if cost < min_cost:
                min_cost = cost
                last_city = k
        self.total_min_cost = min_cost
        self.last_city = last_city

    def reconstruct_path(self) -> List[int]:
        """Reconstructs the optimal path from the computed policy."""
        subset = (1 << self.n_cities) - 1
        path = [0]
        last_city = self.last_city
        for _ in range(self.n_cities - 1):
            path.append(last_city)
            subset ^= (1 << last_city)
            last_city = self.route_policy.get((subset | (1 << last_city), last_city))
            if last_city is None:
                break
        path.append(0)  # Return to the starting city
        path.reverse()
        return path

    def get_action(self, current_city: int, visited_cities: np.ndarray) -> int:
        """Determines the next city to visit based on the policy."""
        subset = sum(1 << i for i in range(self.n_cities) if visited_cities[i])
        if subset == (1 << self.n_cities) - 1:
            return 0  # Return to starting city if all visited

        min_cost = np.inf
        next_city = None
        for city in range(self.n_cities):
            if not visited_cities[city]:
                s = subset | (1 << city)
                if (s, city) in self.cost_table:
                    cost = self.cost_table[(s, city)] + self.dist_matrix[city, 0]
                    if cost < min_cost:
                        min_cost = cost
                        next_city = city
        if next_city is not None:
            return next_city
        else:
            # If policy is undefined, select the nearest unvisited city
            unvisited = [i for i in range(self.n_cities) if not visited_cities[i]]
            return min(unvisited, key=lambda x: self.dist_matrix[current_city, x])

In [2]:
# compute_solutionrs/monte_carlo_epsilon_greedy_compute_solutionr.py

import numpy as np
import random
from typing import List, Tuple, Dict


class MonteCarloEpsilonGreedySolver:
    """
    Monte Carlo compute_solutionr with epsilon-greedy policy for the TSP.
    Balances exploration and exploitation during policy learning.
    """

    def __init__(self, n_cities: int, dist_matrix: np.ndarray, epsilon: float = 0.1):
        self.n_cities = n_cities
        self.dist_matrix = dist_matrix
        self.epsilon = epsilon
        self.Q_values = {}
        self.returns = {}
        self.policy = {}

    def epsilon_greedy(self, state: Tuple[int, int], possible_actions: List[int]) -> int:
        """Selects an action using epsilon-greedy strategy."""
        if random.random() < self.epsilon:
            return random.choice(possible_actions)
        else:
            q_values = [(action, self.Q_values.get((state, action), float('-inf'))) for action in possible_actions]
            max_q = max(q_values, key=lambda x: x[1])[1]
            best_actions = [action for action, q in q_values if q == max_q]
            return random.choice(best_actions)

    def generate_episode(self) -> List[Tuple[Tuple[int, int], int, float]]:
        """Generates an episode using the epsilon-greedy policy."""
        episode = []
        visited = np.zeros(self.n_cities, dtype=bool)
        current_city = random.randint(0, self.n_cities - 1)
        visited[current_city] = True

        while not visited.all():
            possible_actions = [i for i in range(self.n_cities) if not visited[i]]
            state_bitmask = self._state_to_bitmask(visited)
            state = (current_city, state_bitmask)
            action = self.epsilon_greedy(state, possible_actions)
            reward = -self.dist_matrix[current_city, action]
            episode.append((state, action, reward))
            current_city = action
            visited[current_city] = True

        return episode

    def _state_to_bitmask(self, visited: np.ndarray) -> int:
        """Converts the visited cities array to a bitmask."""
        return sum(1 << i for i, v in enumerate(visited) if v)

    def compute_solution(self, num_episodes: int = 10000, method: str = "first_visit"):
        """Trains the policy using Monte Carlo simulations with epsilon-greedy exploration."""
        for _ in range(num_episodes):
            episode = self.generate_episode()
            G = 0
            visited_state_actions = set()

            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G += reward

                if method == "first_visit" and (state, action) in visited_state_actions:
                    continue

                visited_state_actions.add((state, action))
                if (state, action) not in self.returns:
                    self.returns[(state, action)] = []
                self.returns[(state, action)].append(G)
                self.Q_values[(state, action)] = np.mean(self.returns[(state, action)])

                # Update policy
                current_city, _ = state
                possible_actions = [a for a in range(self.n_cities) if a != current_city]
                self.policy[state] = max(
                    possible_actions,
                    key=lambda a: self.Q_values.get((state, a), float('-inf'))
                )

    def get_action(self, current_city: int, visited_cities: np.ndarray) -> int:
        """Selects the next city to visit using the epsilon-greedy policy."""
        possible_actions = [i for i in range(self.n_cities) if not visited_cities[i]]
        if not possible_actions:
            return 0  # Return to starting city if all cities are visited

        state_bitmask = self._state_to_bitmask(visited_cities)
        state = (current_city, state_bitmask)
        return self.epsilon_greedy(state, possible_actions)

In [3]:
# compute_solutionrs/monte_carlo_compute_solutionr.py

import numpy as np
import random
from typing import List, Tuple, Dict


class MonteCarloSolver:
    """
    Monte Carlo compute_solutionr for the TSP.
    Learns a policy by simulating episodes and updating value estimates.
    """

    def __init__(self, n_cities: int, dist_matrix: np.ndarray):
        self.n_cities = n_cities
        self.dist_matrix = dist_matrix
        self.Q_values = {}
        self.returns = {}
        self.policy = {}

    def generate_episode(self) -> List[Tuple[Tuple[int, int], int, float]]:
        """Generates an episode by randomly selecting actions."""
        episode = []
        visited = np.zeros(self.n_cities, dtype=bool)
        current_city = random.randint(0, self.n_cities - 1)
        visited[current_city] = True

        while not visited.all():
            possible_actions = [i for i in range(self.n_cities) if not visited[i]]
            action = random.choice(possible_actions)
            reward = -self.dist_matrix[current_city, action]
            state = (current_city, self._state_to_bitmask(visited))
            episode.append((state, action, reward))
            current_city = action
            visited[current_city] = True

        return episode

    def _state_to_bitmask(self, visited: np.ndarray) -> int:
        """Converts the visited cities array to a bitmask."""
        return sum(1 << i for i, v in enumerate(visited) if v)

    def compute_solution(self, num_episodes: int = 10000, method: str = "first_visit"):
        """Trains the policy using Monte Carlo simulations."""
        for _ in range(num_episodes):
            episode = self.generate_episode()
            G = 0
            visited_state_actions = set()

            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G += reward

                if method == "first_visit" and (state, action) in visited_state_actions:
                    continue

                visited_state_actions.add((state, action))
                if (state, action) not in self.returns:
                    self.returns[(state, action)] = []
                self.returns[(state, action)].append(G)
                self.Q_values[(state, action)] = np.mean(self.returns[(state, action)])

                # Update policy
                current_city, _ = state
                possible_actions = [a for a in range(self.n_cities) if a != current_city]
                self.policy[state] = max(
                    possible_actions,
                    key=lambda a: self.Q_values.get((state, a), float('-inf'))
                )

    def get_action(self, current_city: int, visited_cities: np.ndarray) -> int:
        """Selects the next city to visit based on the learned policy."""
        state_bitmask = self._state_to_bitmask(visited_cities)
        state = (current_city, state_bitmask)
        possible_actions = [i for i in range(self.n_cities) if not visited_cities[i]]
        if state in self.policy and self.policy[state] in possible_actions:
            return self.policy[state]
        elif possible_actions:
            return min(possible_actions, key=lambda x: self.dist_matrix[current_city, x])
        else:
            return 0  # Return to starting city if no unvisited cities left

In [4]:
# tsp_env.py

import numpy as np
import gym
from gym import spaces
from typing import Optional, Tuple, List, Dict


class TSPEnv(gym.Env):
    """
    Custom OpenAI Gym environment for the Traveling Salesman Problem (TSP).
    The goal is to find the shortest possible route that visits each city exactly once.
    """

    metadata = {'render.modes': ['human']}

    def __init__(self, n_cities: int, area_size: int = 30, seed: Optional[int] = None):
        super(TSPEnv, self).__init__()
        self.n_cities = n_cities
        self.area_size = area_size
        self.seed(seed)

        # Generate random positions for the cities
        self.city_positions = self._generate_city_positions()

        # Calculate the distance matrix
        self.dist_matrix = self._calculate_dist_matrix()

        # Define action and observation spaces
        self.action_space = spaces.Discrete(self.n_cities)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(self.n_cities),  # Current city
            spaces.MultiBinary(self.n_cities)  # Visited cities
        ))

        self.current_city = 0
        self.visited_cities = np.zeros(self.n_cities, dtype=np.int8)
        self.total_distance = 0.0

    def _generate_city_positions(self) -> np.ndarray:
        """Generates random (x, y) positions for each city within the specified area."""
        return np.random.uniform(0, self.area_size, size=(self.n_cities, 2))

    def _calculate_dist_matrix(self) -> np.ndarray:
        """Calculates the Euclidean distance between all pairs of cities."""
        num = self.n_cities
        dist_matrix = np.zeros((num, num))
        for i in range(num):
            for j in range(num):
                if i != j:
                    dist_matrix[i, j] = np.linalg.norm(self.city_positions[i] - self.city_positions[j])
        return dist_matrix

    def reset(self) -> Tuple[int, np.ndarray]:
        """Resets the environment to the initial state."""
        self.current_city = 0
        self.visited_cities = np.zeros(self.n_cities, dtype=np.int8)
        self.visited_cities[self.current_city] = 1
        self.total_distance = 0.0
        return self.current_city, self.visited_cities.copy()

    def step(self, action: int) -> Tuple[Tuple[int, np.ndarray], float, bool, bool, Dict]:
        """Performs the action of moving to the next city."""
        assert self.action_space.contains(action), f"Invalid action: {action}"

        reward = 0.0
        done = False

        if self.visited_cities[action]:
            # High penalty for revisiting a city
            reward = -10000.0
            done = True  # End the episode on invalid action
        else:
            # Calculate distance to the next city
            distance = self.dist_matrix[self.current_city, action]
            self.total_distance += distance
            reward = -distance  # Negative reward for traveling distance
            self.current_city = action
            self.visited_cities[action] = 1

            # Check if all cities have been visited
            if self.visited_cities.sum() == self.n_cities:
                done = True

        observation = (self.current_city, self.visited_cities.copy())
        info = {}

        return observation, reward, done, False, info

    def render(self, mode='human'):
        """Renders the environment (not implemented)."""
        pass

    def seed(self, seed: Optional[int] = None):
        """Sets the random seed for reproducibility."""
        if seed is not None:
            np.random.seed(seed)

In [5]:

def initialize_dp_compute_solutionr(env: TSPEnv) -> DPSolver:
    """Creates and compute_solutions a Dynamic Programming compute_solutionr for the TSP environment."""
    compute_solutionr = DPSolver(env.n_cities, env.dist_matrix)
    compute_solutionr.compute_solution()
    return compute_solutionr


def create_monte_carlo_compute_solutionr(env: TSPEnv, num_episodes: int = 10000, method: str = "first_visit") -> MonteCarloSolver:
    """Creates and trains a Monte Carlo compute_solutionr for the TSP environment."""
    compute_solutionr = MonteCarloSolver(env.n_cities, env.dist_matrix)
    compute_solutionr.compute_solution(num_episodes, method)
    return compute_solutionr


def create_monte_carlo_epsilon_greedy_compute_solutionr(env: TSPEnv, num_episodes: int = 10000, epsilon: float = 0.1, method: str = "first_visit") -> MonteCarloEpsilonGreedySolver:
    """Creates and trains a Monte Carlo Epsilon-Greedy compute_solutionr for the TSP environment."""
    compute_solutionr = MonteCarloEpsilonGreedySolver(env.n_cities, env.dist_matrix, epsilon)
    compute_solutionr.compute_solution(num_episodes, method)
    return compute_solutionr

In [6]:
# main.py

def main():
    n_cities = 6
    num_episodes = 100

    env = TSPEnv(n_cities, seed=42)

    # Create compute_solutionrs
    dp_compute_solutionr = initialize_dp_compute_solutionr(env)
    mc_compute_solutionr = create_monte_carlo_compute_solutionr(env, num_episodes=10000, method="first_visit")
    mc_epsilon_greedy_compute_solutionr = create_monte_carlo_epsilon_greedy_compute_solutionr(env, num_episodes=10000, epsilon=0.1, method="first_visit")

    compute_solutionrs = {
        "Dynamic Programming": dp_compute_solutionr,
        "Monte Carlo": mc_compute_solutionr,
        "Monte Carlo Epsilon-Greedy": mc_epsilon_greedy_compute_solutionr
    }

    total_performance = {}

    for compute_solutionr_name, compute_solutionr in compute_solutionrs.items():
        print(f"\nTesting {compute_solutionr_name}:")
        episode_returns = []

        for episode in range(num_episodes):
            observation = env.reset()
            done = False
            total_reward = 0
            current_city, visited_cities = observation

            while not done:
                action = compute_solutionr.get_action(current_city, visited_cities)
                observation, reward, done, _, _ = env.step(action)
                total_reward += reward
                current_city, visited_cities = observation

            episode_returns.append(total_reward)
            print(f"Episode {episode}: Total Reward = {total_reward}")

        avg_return = np.mean(episode_returns)
        print(f"Average return over {num_episodes} episodes: {avg_return}")
        total_performance[compute_solutionr_name] = avg_return

    print("\nPerformance Comparison:")
    for compute_solutionr_name, avg_return in total_performance.items():
        print(f"{compute_solutionr_name} Average Return: {avg_return}")


if __name__ == "__main__":
    main()


Testing Dynamic Programming:
Episode 0: Total Reward = -59.15168247679169
Episode 1: Total Reward = -59.15168247679169
Episode 2: Total Reward = -59.15168247679169
Episode 3: Total Reward = -59.15168247679169
Episode 4: Total Reward = -59.15168247679169
Episode 5: Total Reward = -59.15168247679169
Episode 6: Total Reward = -59.15168247679169
Episode 7: Total Reward = -59.15168247679169
Episode 8: Total Reward = -59.15168247679169
Episode 9: Total Reward = -59.15168247679169
Episode 10: Total Reward = -59.15168247679169
Episode 11: Total Reward = -59.15168247679169
Episode 12: Total Reward = -59.15168247679169
Episode 13: Total Reward = -59.15168247679169
Episode 14: Total Reward = -59.15168247679169
Episode 15: Total Reward = -59.15168247679169
Episode 16: Total Reward = -59.15168247679169
Episode 17: Total Reward = -59.15168247679169
Episode 18: Total Reward = -59.15168247679169
Episode 19: Total Reward = -59.15168247679169
Episode 20: Total Reward = -59.15168247679169
Episode 21: To